# Document Classification with Naive Bayes - Lab

## Introduction

In this lecture, you'll practice implementing the Naive Bayes algorithm on your own.

## Objectives

You will be able to:  

* Implement document classification using Naive Bayes
* Understand the need for the Laplacian smoothing correction
* Explain how to code a bag of words representation

## Import the Dataset

To start, import the dataset stored in the text file `SMSSpamCollection`.

In [7]:
import numpy as np
import pandas as pd
spam = pd.read_csv('SMSSpamCollection', sep='\t', names=['label', 'text'])
spam.head(1)

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."


## Account for Class Imbalance

To help your algorithm perform more accurately, subset the dataset so that the two classes are of equal size. To do this, keep all of the instances of the minority class (spam) and subset examples of the majority class (ham) to an equal number of examples.

In [29]:
print(spam[spam['label']=='ham'].count())
print(spam[spam['label']=='spam'].count())

label    4825
text     4825
dtype: int64
label    747
text     747
dtype: int64


In [14]:
majority_sample = spam[spam.label=='ham'].sample(n=len(minority))
len(majority_sample)

747

In [31]:
spam_balanced = pd.concat([minority, majority_sample])
len(spam_balanced)

1494

## Train - Test Split

Now implement a train test split on your dataset.

In [32]:
from sklearn.model_selection import train_test_split
X = spam_balanced.text
y = spam_balanced.label
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)
train_spam_balanced = pd.concat([X_train, y_train], axis=1)
test_spam_balanced = pd.concat([X_test, y_test], axis=1)

## Create the Word Frequency Dictionary for Each Class

Create a word frequency dictionary for each class.

In [33]:
class_word_freq = {} 
classes = train_spam_balanced.label.unique()
for class_ in classes:
    temp_spam = train_spam_balanced[train_spam_balanced.label==class_]
    bag = {}
    for row in temp_spam.index:
        doc = temp_spam['text'][row]
        for word in doc.split():
            bag[word] = bag.get(word, 0) + 1
    class_word_freq[class_] = bag

## Count the Total Corpus Words
Calculate V, the total number of words in the corpus.

In [34]:
vocabulary = set()
for text in train_spam_balanced.text:
    for word in text.split():
        vocabulary.add(word)
V = len(vocabulary)
V

6001

## Create a Bag of Words Function

Before implementing the entire Naive Bayes algorithm, create a helper function `bag_it()` to create a bag of words representation from a document's text.

In [35]:
def bag_it(doc):
    bag = {}
    for word in doc.split():
        bag[word] = bag.get(word, 0) + 1
    return bag

## Implementing Naive Bayes

Now, implement a master function to build a naive Bayes classifier. Be sure to use the logarithmic probabilities to avoid underflow.

In [38]:
def classify_doc(doc, class_word_freq, p_classes, V, return_posteriors=False):
    bag = bag_it(doc)
    classes = []
    posteriors = []
    for class_ in class_word_freq.keys():
        p = np.log(p_classes[class_])
        for word in bag.keys():
            num = bag[word]+1
            denom = class_word_freq[class_].get(word, 0) + V
            p += np.log(num/denom)
        classes.append(class_)
        posteriors.append(p)
    if return_posteriors:
        print(posteriors)
    return classes[np.argmax(posteriors)]

## Test Out Your Classifier

Finally, test out your classifier and measure its accuracy. Don't be perturbed if your results are sub-par; industry use cases would require substantial additional preprocessing before implementing the algorithm in practice.

In [39]:
p_classes = dict(spam_balanced.label.value_counts(normalize=True))
p_classes
y_hat_train = X_train.map(lambda x: classify_doc(x, class_word_freq, p_classes, V))
residuals = y_train == y_hat_train
residuals.value_counts(normalize=True)

False    0.763393
True     0.236607
dtype: float64

## Level-Up

Rework your code into an appropriate class structure so that you could easily implement the algorithm on any given dataset.

## Summary

Well done! In this lab, you practiced implementing Naive Bayes for document classification!